In [29]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers
from tqdm import trange

In [72]:
df = pd.read_csv('data/single_step_df_ints_2022-06-03_encoded.csv', header=0)

In [68]:
df_new[1436:]

,input,target,start_coords.x,start_coords.y,start_coords.z,p.already_seen,n.already_seen,w.already_seen,t.already_seen,g.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
1436,<start>,i,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1437,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1438,k,d,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1439,d,p,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1440,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2069,d,i,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2070,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2071,k,x,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [73]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [74]:
df['target'].unique()
df_new[:1435]['target'].value_counts()
#df[:1437]['target'].loc[df[:1437]['target'] == 'i'].index[0]
#df_new[1436:]['target'].value_counts()

d    137
k    116
i    105
p     61
u     56
q     43
b     32
v     30
f     15
r     13
s      8
g      8
x      5
c      4
a      2
y      1
h      1
Name: target, dtype: int64

In [75]:
#index_to_drop = df[1436:]['target'].loc[df[1436:]['target'] == 'y'].index[0]
#index_to_drop2 = df[1436:]['target'].loc[df[1436:]['target'] == 'h'].index[0]
index_to_drop = df[:1435]['target'].loc[df[:1435]['target'] == 'y'].index[0]
index_to_drop

(1945, 1958)

In [76]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [77]:
# remove row with target that only occurs once

df_new = df.drop(index=[index_to_drop, 727], axis=0)
#df_new = df.drop(index=[index_to_drop, index_to_drop2], axis=0)
df_new.reset_index(inplace=True, drop=True)

In [78]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new[:1435], df_new[:1435]['target']):
#for train_index, test_index in split.split(df_new, df_new['target']):
#for train_index, test_index in split.split(df_new[1436:], df_new[1436:]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [79]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [80]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [81]:
len(labels)

23

In [15]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [16]:
batch_size = 128

In [82]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_18044/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [18]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [19]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [20]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [21]:
all_inputs, encoded_features = create_input_data(df)

In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [23]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [24]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [83]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
4/4 [==============================] - 0s 46ms/step - loss: 1.2524 - accuracy: 0.5753 - val_loss: 0.8404 - val_accuracy: 0.7500
Epoch 2/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1802 - accuracy: 0.5775 - val_loss: 0.8513 - val_accuracy: 0.7604
Epoch 3/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1627 - accuracy: 0.5933 - val_loss: 0.8720 - val_accuracy: 0.7604
Epoch 4/100
4/4 [==============================] - 0s 44ms/step - loss: 1.1465 - accuracy: 0.6000 - val_loss: 0.8834 - val_accuracy: 0.7500
Epoch 5/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1316 - accuracy: 0.5798 - val_loss: 0.8862 - val_accuracy: 0.7500
Epoch 6/100
4/4 [==============================] - 0s 47ms/step - loss: 1.0938 - accuracy: 0.6270 - val_loss: 0.8808 - val_accuracy: 0.7292
Epoch 7/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1342 - accuracy: 0.5865 - val_loss: 0.8724 - val_accuracy: 0.7292
Epoch 8/100
4/4 [===

In [84]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 69ms/step - loss: 1.0930 - accuracy: 0.6526
Accuracy 0.6526315808296204


In [50]:
model.save('models/next_obj_classifier_tablesetting_2022-06-07')

2022-06-07 10:03:41.886830: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


In [51]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_tablesetting_2022-06-07/')

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [30]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in trange(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [53]:
df[1438:].index[-1]

2073

In [85]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

100%|████████████████████████████████████████████████████████████| 2073/2073 [12:07<00:00,  2.85it/s]


In [88]:
summed_error = [sum(error) for error in errors[:-1]]

In [89]:
np.median(summed_error)

3.0

In [57]:
with open('results/nn_spatialinfo_prequential_summed_fitted_on_ts_results_all_2022-06-07.txt', 'w') as file:
    file.write(str(summed_error))

In [58]:
np.mean(summed_error)

3.466019417475728

In [34]:
results = pd.DataFrame()

In [90]:
results['fitted_on_cooking_all'] = summed_error

In [49]:
df_ids = pd.read_csv('../../opportunistic_planning/all_lowest_error/all_task_environments_list_2022-06-02_unique_sequences.csv')

In [91]:
results

,fitted_on_ts_all,fitted_on_all_all,ID,fitted_on_cooking_all
0,1.0,1.0,a1,2.0
1,4.0,3.0,a3,2.0
2,3.0,3.0,a5,3.0
3,8.0,8.0,a11,8.0
4,2.0,0.0,a13,1.0
...,...,...,...,...
304,6.0,5.0,s37-d21,5.0
305,3.0,3.0,s37-d25,3.0
306,4.0,3.0,s37-d29,3.0
307,8.0,6.0,s37-d39,6.0


In [93]:
results.to_csv('results/nn_spatialinfo_prequential_fitting_variants_2022-06-23.csv', header=True, index=False)